In [5]:
# !pip install spacy-streamlit>=1.0.0a0
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.3.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines        en_core_web_trf (3.3.0), en_core_web_sm (3.3.0)



In [9]:
import logging
import os
import re
from collections import OrderedDict
from typing import List
from pathlib import Path
import random


import pandas as pd
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example

ENTITY_TAGS = ["CVE",
               "CWE",
               "SOFTWARE",
               "MALWARE",
               "COURSE_OF_ACTION",
               "INTRUSION_SET",
               "THREAT_ACTOR",
               "TOOL",
               "ATTACK_PATTERN",
               "INDUSTRY",
               "MITRE_ATTACK",
               "CAMPAIGN",
               "ORG",
               "COUNTRY",
               "CITY",
               "GEOLOCATION",
               "TIMESTAMP",
               ]

In [7]:
nlp = spacy.blank("en")

In [8]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe('ner')

In [10]:
for ent in ENTITY_TAGS:
    ner.add_label(ent)

In [11]:
df = pd.read_csv(r'/content/entities_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4541 entries, 0 to 4540
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  4541 non-null   object
 1   tag       4541 non-null   object
 2   entity    4541 non-null   object
dtypes: object(3)
memory usage: 106.6+ KB


In [19]:
TRAINING_DATA = []
for _, row in df.iterrows():
  start_idx = 0
  end_idx = len(row['entity'])-1
  entity_info = (start_idx, end_idx, row['tag'])
  TRAINING_DATA.append(
      (row['entity'], 
       dict(entities=[entity_info]) )
      )
  
TRAINING_DATA

[('CVE-2011-3544', {'entities': [(0, 12, 'CVE')]}),
 ('CVE 2013-2465', {'entities': [(0, 12, 'CVE')]}),
 ('CVE-2013-2465', {'entities': [(0, 12, 'CVE')]}),
 ('CVE-2012-4681', {'entities': [(0, 12, 'CVE')]}),
 ('CVE-2012-4681', {'entities': [(0, 12, 'CVE')]}),
 ('CVE-2013-2465', {'entities': [(0, 12, 'CVE')]}),
 ('CVE-2013-2460', {'entities': [(0, 12, 'CVE')]}),
 ('PowerShell', {'entities': [(0, 9, 'SOFTWARE')]}),
 ('PowerShell', {'entities': [(0, 9, 'SOFTWARE')]}),
 ('SNORT', {'entities': [(0, 4, 'SOFTWARE')]}),
 ('ClamAV', {'entities': [(0, 5, 'SOFTWARE')]}),
 ('Java', {'entities': [(0, 3, 'SOFTWARE')]}),
 ('Java', {'entities': [(0, 3, 'SOFTWARE')]}),
 ('PowerShell', {'entities': [(0, 9, 'SOFTWARE')]}),
 ('PowerShell', {'entities': [(0, 9, 'SOFTWARE')]}),
 ('Java', {'entities': [(0, 3, 'SOFTWARE')]}),
 ('Windows API', {'entities': [(0, 10, 'SOFTWARE')]}),
 ('Snort', {'entities': [(0, 4, 'SOFTWARE')]}),
 ('Snort', {'entities': [(0, 4, 'SOFTWARE')]}),
 ('Snort', {'entities': [(0, 4, 'SO

In [22]:
from spacy.training.example import Example
import random

disable_pipes =[pipe for pipe in nlp.pipe_names if pipe != 'ner']
iter_num = 100
with nlp.disable_pipes(*disable_pipes):
    optimizer = nlp.begin_training()
    for itr in range(iter_num):
        try:
            random.shuffle(TRAINING_DATA)
            losses = {}

            for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
              for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                
                nlp.update([example],
                          drop = 0.35,  
                          sgd = optimizer,
                          losses = losses,
                           )
            
            print('Interation = '+str(itr))
            print('Losses = '+str(losses))
        
        except ValueError as ve:
          print(ve)

        

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Shanghai" with entities "[(0, 7, 'CITY')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Mandiant" with entities "[(0, 7, 'ORG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Geneva" with entities "[(0, 5, 'CITY')]". Use `

Interation = 0
Losses = {'ner': 107.35786447894196}
Interation = 1
Losses = {'ner': 118.95593579776765}
Interation = 2
Losses = {'ner': 98.89673577778979}
Interation = 3
Losses = {'ner': 126.45607040224472}
Interation = 4
Losses = {'ner': 101.71236097249489}
Interation = 5
Losses = {'ner': 133.29469190227746}
Interation = 6
Losses = {'ner': 101.09578476627141}
Interation = 7
Losses = {'ner': 81.2203013907696}
Interation = 8
Losses = {'ner': 98.22695884824117}
Interation = 9
Losses = {'ner': 81.75280832842861}
Interation = 10
Losses = {'ner': 96.28049232282807}
Interation = 11
Losses = {'ner': 63.67691299889594}
Interation = 12
Losses = {'ner': 77.30017718896218}
Interation = 13
Losses = {'ner': 72.62129821777506}
Interation = 14
Losses = {'ner': 85.21931895175685}
Interation = 15
Losses = {'ner': 67.66888121136516}
Interation = 16
Losses = {'ner': 73.17878369518391}
Interation = 17
Losses = {'ner': 69.0218964809122}
Interation = 18
Losses = {'ner': 61.767515838365846}
Interation = 19
L

In [24]:
from pathlib import Path

output_dir=Path("/content/ner_data/")
nlp.to_disk(output_dir)
print(f"Saved model to {output_dir}") 

Saved model to /content/ner_data


In [26]:
# Загрузка лучшей модели
loaded_nlp = spacy.load(r"/content/ner_data/") 

In [42]:
# Эмпирическая проверка работы обученной модели
# Проверка проводится на выжимке из текста, представленного в наборе данных
test_sample = "Windows API"
doc = loaded_nlp(test_sample) 

# Визуализация найденных сущностей
spacy.displacy.render(doc, style="ent", jupyter=True) 

In [43]:
doc = loaded_nlp("Cobalt") 
spacy.displacy.render(doc, style="ent", jupyter=True)